### Imports

In [54]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Scraper Class

Contains the BASE URL and the methods that allows to extract the information from the UK Open Data

In [133]:
class DataScraper():
    
    BASE_URL = "https://www.data.gov.uk/dataset/e7d4c1cf-45a0-4070-878f-24ad9641f655/domestic-electricity-and-gas-estimates-by-postcode-in-great-britain"
    
    def __init__(self):
        self.electricity_elements = []
        self.csv_elements = []
        self.df = None
        
    def get_electricity_links(self):
        """
            Allows to get all the elements of the table that contains electricity, with their corresponding
            links
        """
        res = requests.get(self.BASE_URL)
        soup = BeautifulSoup(res.content)
        cell_elements = soup.find_all("td", {"class": "govuk-table__cell"})
        electricity_elements = []
        for cell in cell_elements:
            [s.extract() for s in cell('span')] # Extract the span elements to remove extra text
            text_content = ' '.join(cell.strings).replace("\n", "").strip()
            if "electricity" in text_content:
                link = cell.find("a")["href"]
                electricity_elements.append({"text": text_content, "link": link})
        
        self.electricity_elements = electricity_elements
        return electricity_elements
    
    def get_electricity_csv(self):
        """
            Allows to get, from the links obtained with get_electricity_links(), the .csv files with
            the data
        """
        
        csv_elements = []
        for el in self.electricity_elements:
            text = el["text"]
            link = el["link"]
            year = text.split(" ")[0].strip() # The years is shown always before the word electricity

            # I'll skip the year 2013, because there is no information for 2014
            if year == "2013":
                continue

            # If the link is already to the csv file
            if "uploads" in link:
                csv_elements.append({
                    "text": text,
                    "link": link,
                    "year": year
                })
                continue


            else: # bring the page of statistics and get the .csv link
                res = requests.get(link)
                soup = BeautifulSoup(res.content)

                sections = soup.find("section", {"id": "documents"}).find_all("section")
                # Select the section that contains the words "level all"
                all_level_section = [sections[idx] for idx, section in enumerate(sections) if
                                    "level all" in ' '.join(section.strings)][0]

                # The csv link is in the last element of the a tags
                csv_elements.append({
                    "text": text,
                    "link": all_level_section.find_all("a")[-1]["href"],
                    "year": year
                })
                
        self.csv_elements = csv_elements
        
        return csv_elements
    
    def get_df(self):
        """
            Returns a Pandas DataFrame that contains all the elements of the CSV, with their corresponding
            year
        """
        
        output_list = []
        for el in self.csv_elements:
            year = el["year"]
            link = el["link"]
            print(f"Reading the year {year}, link: {link}")
            df_tmp = pd.read_csv(link)
            df_tmp["Year"] = year
            output_list.append(df_tmp)
        print("DONE!")    
        
        df_electricity = pd.concat(output_list, ignore_index = True)
        
        self.df = df_electricity
        
        return df_electricity
    
    def save_df(self, path):
        print(f"Saving .CSV file in {path}")
        self.df.to_csv(path, index=False)

### Testing

In [129]:
scraper = DataScraper()

Get electricity links

In [130]:
scraper.get_electricity_links()

[{'text': '2021 electricity',
  'link': 'https://www.gov.uk/government/statistics/postcode-level-electricity-statistics-2021-experimental'},
 {'text': '2020 electricity',
  'link': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1050243/Postcode_level_all_meters_electricity_2020.csv'},
 {'text': '2019 electricity',
  'link': 'https://www.gov.uk/government/statistics/postcode-level-electricity-statistics-2019-experimental'},
 {'text': '2018 electricity',
  'link': 'https://www.gov.uk/government/statistics/postcode-level-electricity-statistics-2018-experimental'},
 {'text': '2017 electricity',
  'link': 'https://www.gov.uk/government/statistics/postcode-level-electricity-statistics-2017-experimental'},
 {'text': '2016 electricity',
  'link': 'https://www.gov.uk/government/statistics/postcode-level-electricity-statistics-2016-experimental'},
 {'text': '2015 electricity',
  'link': 'https://www.gov.uk/government/statistics/postcode-level-ele

Get csv links

In [131]:
scraper.get_electricity_csv()

[{'text': '2021 electricity',
  'link': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1131250/Postcode_level_all_meters_electricity_2021.csv',
  'year': '2021'},
 {'text': '2020 electricity',
  'link': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1050243/Postcode_level_all_meters_electricity_2020.csv',
  'year': '2020'},
 {'text': '2019 electricity',
  'link': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1131264/Postcode_level_all_meters_electricity_2019.csv',
  'year': '2019'},
 {'text': '2018 electricity',
  'link': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1131268/Postcode_level_all_meters_electricity_2018.csv',
  'year': '2018'},
 {'text': '2017 electricity',
  'link': 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1131277/Pos

Get DataFrame

In [132]:
scraper.get_df()

Reading the year 2021, link: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1131250/Postcode_level_all_meters_electricity_2021.csv
Reading the year 2020, link: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1050243/Postcode_level_all_meters_electricity_2020.csv
Reading the year 2019, link: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1131264/Postcode_level_all_meters_electricity_2019.csv
Reading the year 2018, link: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1131268/Postcode_level_all_meters_electricity_2018.csv
Reading the year 2017, link: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1131277/Postcode_level_all_meters_electricity_2017.csv
Reading the year 2016, link: https://assets.publishing.service.gov.uk/government/uploads/system/uplo

,Outcode,Postcode,Num_meters,Total_cons_kwh,Mean_cons_kwh,Median_cons_kwh,Year
0,AB10,AB10 1AU,36,100211.80,2783.661111,2118.20,2021
1,AB10,AB10 1BA,30,113706.50,3790.216667,2432.60,2021
2,AB10,AB10 1BB,5,13142.50,2628.500000,1579.00,2021
3,AB10,AB10 1FG,23,45264.50,1968.021739,1556.80,2021
4,AB10,AB10 1FL,56,174750.29,3120.540893,2661.60,2021
...,...,...,...,...,...,...,...
5253782,ZE3,ZE3 9JU,38,197883.90,5207.471053,3976.70,2015
5253783,ZE3,ZE3 9JW,30,241897.70,8063.256667,6794.00,2015
5253784,ZE3,ZE3 9JX,26,145938.20,5613.007692,4452.55,2015
5253785,ZE3,ZE3 9JY,22,115758.30,5261.740909,4805.05,2015
